# Brute-Force Portfolio Optimization

This notebook implements an exhaustive search method to solve the portfolio optimization problem with cardinality constraint.

## Methodology

The algorithm evaluates **all** possible combinations of B assets selected from a total of n assets:

$$\text{Number of combinations} = \binom{n}{B} = \frac{n!}{B!(n-B)!}$$

For our case: $\binom{21}{4} = 5,985$ combinations

## Advantages
- ✅ **Guarantees finding the global optimum**
- ✅ Simple and verifiable implementation
- ✅ Useful as baseline for comparing other methods

## Limitations
- ❌ **Factorial complexity** O(n choose B)
- ❌ Impractical for n > 25 or B > 10
- ❌ Does not scale to real-world problems (e.g., 500 assets)

---

In [42]:
import numpy as np
from itertools import combinations
import time
import math

print("✓ Libraries imported successfully")


✓ Libraries imported successfully


## 1. Load QUBO Data

Load the QUBO formulation generated in `Data-and-QUBO.ipynb`:
- **Q**: Quadratic matrix (21×21) encoding scaled covariance
- **q**: Linear vector (21,) encoding expected returns
- **Σ**: Covariance matrix of daily returns
- **μ**: Vector of expected daily returns
- **B**: Cardinality (number of assets to select)


In [43]:
# Load QUBO data
data = np.load("portfolio_qubo_data.npz", allow_pickle=True)

Q = data['Q']
q = data['q']
mu = data['mu']
Sigma = data['Sigma']
B = int(data['B'])
TICKERS = list(data['TICKERS'])
n = len(TICKERS)

print("✓ Data loaded successfully.")
print(f"  n = {n} assets")
print(f"  B = {B} cardinality")
print(f"  Q shape: {Q.shape}")
print(f"  q shape: {q.shape}")
print(f"  Total combinations to evaluate: {math.comb(n, B):,}")


✓ Data loaded successfully.
  n = 21 assets
  B = 4 cardinality
  Q shape: (21, 21)
  q shape: (21,)
  Total combinations to evaluate: 5,985


## 2. Define Evaluation Functions

### QUBO Objective Function
$$f(x) = x^T Q x + q^T x$$

where $x \in \{0,1\}^n$ is the binary selection vector.

### Financial Metrics
For an equal-weighted portfolio with selected assets:
- **Annualized return**: $\mu_{ann} = 252 \cdot \mu^T w$
- **Annualized volatility**: $\sigma_{ann} = \sqrt{252 \cdot w^T \Sigma w}$
- **Sharpe Ratio**: $SR = \mu_{ann} / \sigma_{ann}$

where $w_i = 1/B$ if asset $i$ is selected, 0 otherwise.

In [44]:
# Load QUBO data
data = np.load("portfolio_qubo_data.npz", allow_pickle=True)

Q = data['Q']
q = data['q']
mu = data['mu']
Sigma = data['Sigma']
B = int(data['B'])
TICKERS = list(data['TICKERS'])
n = len(TICKERS)

print("✓ Data loaded successfully.")
print(f"  n = {n} assets")
print(f"  B = {B} cardinality")
print(f"  Q shape: {Q.shape}")
print(f"  q shape: {q.shape}")
print(f"  Total combinations to evaluate: {math.comb(n, B):,}")


✓ Data loaded successfully.
  n = 21 assets
  B = 4 cardinality
  Q shape: (21, 21)
  q shape: (21,)
  Total combinations to evaluate: 5,985


## 2. Define Evaluation Functions

### QUBO Objective Function
$$f(x) = x^T Q x + q^T x$$

where $x \in \{0,1\}^n$ is the binary selection vector.

### Financial Metrics
For an equal-weighted portfolio with selected assets:
- **Annualized return**: $\mu_{ann} = 252 \cdot \mu^T w$
- **Annualized volatility**: $\sigma_{ann} = \sqrt{252 \cdot w^T \Sigma w}$
- **Sharpe Ratio**: $SR = \mu_{ann} / \sigma_{ann}$

where $w_i = 1/B$ if asset $i$ is selected, 0 otherwise.

In [45]:
def f_qubo(x):
    """
    Calculate QUBO cost for a binary vector x.
    
    Parameters:
    -----------
    x : np.ndarray (n,)
        Binary asset selection vector
    
    Returns:
    --------
    float : QUBO cost = x^T Q x + q^T x
    """
    return float(x @ Q @ x + q @ x)

def metrics_for_indices(sel_idx):
    """
    Calculate financial metrics for selected assets.
    
    Parameters:
    -----------
    sel_idx : array-like
        Indices of selected assets
    
    Returns:
    --------
    tuple : (annualized_return, annualized_volatility)
    """
    w = np.zeros(n)
    w[sel_idx] = 1.0 / B  # Equal-weighted
    
    mu_day = float(mu @ w)
    var_day = float(w @ Sigma @ w)
    
    mu_ann = 252 * mu_day    # 252 trading days
    std_ann = np.sqrt(252 * var_day)
    
    return mu_ann, std_ann

print("✓ Functions defined")


✓ Functions defined


## 3. Exhaustive Search

Iterate over all $\binom{n}{B}$ possible combinations and evaluate the QUBO cost for each, keeping track of the best solution found.


In [46]:
# Initialize
best_val = np.inf
best_x = None

print("\n🔍 Running brute-force search...")
print(f"   Evaluating {math.comb(n, B):,} combinations...")

# Measure execution time
t_start = time.perf_counter()

# Exhaustive search
for comb in combinations(range(n), B):
    x = np.zeros(n)
    x[list(comb)] = 1
    val = f_qubo(x)
    
    if val < best_val:
        best_val = val
        best_x = x.copy()

t_end = time.perf_counter()
t_bruteforce = t_end - t_start

# Calculate financial metrics
sel_idx_BF = np.where(best_x == 1)[0]
mu_ann_BF, std_ann_BF = metrics_for_indices(sel_idx_BF)
sharpe_BF = mu_ann_BF / std_ann_BF

print(f"✓ Search completed in {t_bruteforce:.4f} seconds")

# VERIFICATION: Recalculate cost to confirm
verify_cost = f_qubo(best_x)
print(f"\n🔍 Verification:")
print(f"   Best cost found during search: {best_val:.6f}")
print(f"   Cost recalculated: {verify_cost:.6f}")
print(f"   Match: {np.isclose(best_val, verify_cost)}")



🔍 Running brute-force search...
   Evaluating 5,985 combinations...
✓ Search completed in 0.0325 seconds

🔍 Verification:
   Best cost found during search: 0.147206
   Cost recalculated: 0.147206
   Match: True


## 4. Global Optimum Results

The brute-force method guarantees finding the global optimal solution to the QUBO problem.


In [47]:
print("\n" + "="*60)
print("         BRUTE-FORCE OPTIMIZATION RESULTS")
print("="*60)
print(f"\n📊 Objective Function:")
print(f"   f(x) = {best_val:.6f}")

print(f"\n💼 Selected Portfolio (B={B} assets):")
selected_tickers = [TICKERS[i] for i in sel_idx_BF]
for i, ticker in enumerate(selected_tickers, 1):
    print(f"   {i}. {ticker}")

print(f"\n📈 Financial Metrics:")
print(f"   Annualized Return:     μ = {mu_ann_BF:.6f} ({mu_ann_BF*100:.2f}%)")
print(f"   Annualized Volatility: σ = {std_ann_BF:.6f} ({std_ann_BF*100:.2f}%)")
print(f"   Sharpe Ratio:         SR = {sharpe_BF:.3f}")

print(f"\n⏱️  Computation Time: {t_bruteforce:.4f} seconds")

# DIAGNOSTIC: Show bitstring
print(f"\n🔍 Solution Details:")
print(f"   Binary vector: {best_x.astype(int)}")
print(f"   Sum (cardinality): {int(best_x.sum())}")
print("="*60)



         BRUTE-FORCE OPTIMIZATION RESULTS

📊 Objective Function:
   f(x) = 0.147206

💼 Selected Portfolio (B=4 assets):
   1. AVGO
   2. TSLA
   3. CAT
   4. VZ

📈 Financial Metrics:
   Annualized Return:     μ = 0.028017 (2.80%)
   Annualized Volatility: σ = 0.170303 (17.03%)
   Sharpe Ratio:         SR = 0.165

⏱️  Computation Time: 0.0325 seconds

🔍 Solution Details:
   Binary vector: [0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0]
   Sum (cardinality): 4


## 5. Save Results

Results are saved in `.npz` format to be loaded by the comparison notebook.


In [48]:
# Save results for comparison
np.savez(
    "bruteforce_results.npz",
    fx_bruteforce=np.array(best_val),
    mu_ann_bruteforce=np.array(mu_ann_BF),
    std_ann_bruteforce=np.array(std_ann_BF),
    sharpe_bruteforce=np.array(sharpe_BF),
    t_bruteforce=np.array(t_bruteforce),
    x_bruteforce=best_x.astype(int),
    selected_assets=np.array(selected_tickers, dtype=object)
)

print("\n✓ Results saved to 'bruteforce_results.npz'")

# VERIFICATION: Load and check what was saved
verify_data = np.load("bruteforce_results.npz", allow_pickle=True)
print(f"\n🔍 Verification of saved data:")
print(f"   fx_bruteforce: {float(verify_data['fx_bruteforce']):.6f}")
print(f"   x_bruteforce: {verify_data['x_bruteforce']}")
print(f"   x_bruteforce sum: {verify_data['x_bruteforce'].sum()}")
print(f"   Selected assets: {list(verify_data['selected_assets'])}")

# FINAL CHECK: Recalculate cost from saved data
x_loaded = verify_data['x_bruteforce']
cost_from_loaded = f_qubo(x_loaded)
print(f"\n🔍 Final consistency check:")
print(f"   Cost from saved x: {cost_from_loaded:.6f}")
print(f"   Saved fx value: {float(verify_data['fx_bruteforce']):.6f}")
print(f"   Match: {np.isclose(cost_from_loaded, float(verify_data['fx_bruteforce']))}")

if np.isclose(cost_from_loaded, float(verify_data['fx_bruteforce'])):
    print(f"   ✅ Data saved correctly!")
else:
    print(f"   ❌ WARNING: Saved data inconsistent!")



✓ Results saved to 'bruteforce_results.npz'

🔍 Verification of saved data:
   fx_bruteforce: 0.147206
   x_bruteforce: [0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0]
   x_bruteforce sum: 4
   Selected assets: ['AVGO', 'TSLA', 'CAT', 'VZ']

🔍 Final consistency check:
   Cost from saved x: 0.147206
   Saved fx value: 0.147206
   Match: True
   ✅ Data saved correctly!
